# Text to Image generation and Enriching using Amazon Titan model
We can generate images and enrich existing images from text prompt using a number of models available in Amazon bedrock. Important inference parameters to consider while generating image from a text are,

- **text prompt** - Can be positive and negative
- **seed** - Random seed for reproducibility
- **size** - Height and Width
- **cfg-scale** - Indicates how close the image should for the prompt. Lower number will increase randomness in the generation of the image.

Note - To find the exact parameter names, refer to model documentation.

We are going to generate an image of a boat using Amazon titan model - `amazon.titan-image-generator-v1`

Documentaion - https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html 

## Step 1: Configure AWS environment

In [1]:
# Set env variables using os module .
## NOT RECOMMENDED FOR PRODUCTION DEPLOYMENTS OR TO SHARE ON PUBLIC DOMAIN
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'your key id'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'your secret'

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

## Step 2: Explore and choose the model of your choice
We will use Amazong titan image model - `amazon.titan-image-generator-v1`

In [2]:
# Cretae boto3 client to access bedrock APIs
import boto3

In [3]:
bedrock = boto3.client(service_name='bedrock',
                      region_name='us-east-1')
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'b5b02914-3179-4ccd-896e-8f69cfa8480b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 10 May 2025 07:31:53 GMT',
   'content-type': 'application/json',
   'content-length': '54644',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b5b02914-3179-4ccd-896e-8f69cfa8480b'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

In [4]:
# Get model details
bedrock.get_foundation_model(modelIdentifier='amazon.titan-image-generator-v1')

{'ResponseMetadata': {'RequestId': 'a9493d1a-119f-4df7-90d1-184b9458fee0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 10 May 2025 07:31:53 GMT',
   'content-type': 'application/json',
   'content-length': '540',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a9493d1a-119f-4df7-90d1-184b9458fee0'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1',
  'modelId': 'amazon.titan-image-generator-v1',
  'modelName': 'Titan Image Generator G1',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT', 'IMAGE'],
  'outputModalities': ['IMAGE'],
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}}}

## Step 3: Prepare prompt for the model
As per the documentation, we can use below request format to create IMAGE from TEXT,
```
{
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": "string",      
        "negativeText": "string"
    },
    "imageGenerationConfig": {
        "quality": "standard" | "premium",
        "numberOfImages": int,
        "height": int,
        "width": int,
        "cfgScale": float,
        "seed": int
    }
}
```

In [5]:
# Use json module to construct the requets payload
import json

In [6]:
# Define the prompt for the model.
positive_prompt = """Create a natural photogenetic image of a boat floating on a sandy beach surrounded by coconut trees"""
negative_prompt = """painting, drawing, blury"""

req = json.dumps({
    'taskType': 'TEXT_IMAGE',
    'textToImageParams': {
        'text': positive_prompt,
        'negativeText': negative_prompt
    },
    'imageGenerationConfig': {
        'numberOfImages': 1,
        'height': 1024,
        'width': 1024,
        'cfgScale': 8.0,
        'seed': 0
    }
})
print(req)

{"taskType": "TEXT_IMAGE", "textToImageParams": {"text": "Create a natural photogenetic image of a boat floating on a sandy beach surrounded by coconut trees", "negativeText": "painting, drawing, blury"}, "imageGenerationConfig": {"numberOfImages": 1, "height": 1024, "width": 1024, "cfgScale": 8.0, "seed": 0}}


## Step 4: Create bedrock runtime to invoke the model

In [7]:
bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

## Step 5: Invoke model to get an image generated

In [8]:
response = bedrock_runtime.invoke_model(body=req,modelId='amazon.titan-image-generator-v1')
respose_body = json.loads(response.get('body').read())

## Step 6: Process the response to get the image
Response body
```
{
  "images": [
    "base64-encoded string", 
    ...
  ],
  "error": "string" 
}
```

In [9]:
# Use base64 module to decode the image
import base64

In [10]:
decoded_image = base64.b64decode(respose_body['images'][0])

In [11]:
# If you encounter PIL not found, then install Pillow - works on MacOS
!pip install Pillow

Looking in indexes: https://pypi.python.org/simple


In [12]:
# To export the image as png, lets use PIL.Image
from PIL import Image

In [13]:
with open("boat.png", 'wb') as file:
    file.write(decoded_image)

#Image.open('boat.png')

## Step 7: Enrich image using image variation
Need to get encoded image to create variation. Request format,
```
{
     "taskType": "IMAGE_VARIATION",
     "imageVariationParams": {
         "text": "string",
         "negativeText": "string",
         "images": ["base64-encoded string"],
         "similarityStrength": 0.7,  # Range: 0.2 to 1.0
     },
     "imageGenerationConfig": {
         "quality": "standard" | "premium",
         "numberOfImages": int,
         "height": int,
         "width": int,
         "cfgScale": float
     }
}
```


In [14]:
# Read and encode the image
with open('boat.png', 'rb') as file:
    image_binary = file.read()
    encoded_image = base64.b64encode(image_binary).decode('utf-8')

In [15]:
# Build image variation request
enrich_req = json.dumps({'taskType': 'IMAGE_VARIATION',
     'imageVariationParams': {
         'text': 'keep the image as it is but change the color of sea to blue',
         'negativeText': 'painting, drawing, blury',
         'images': [encoded_image],
         'similarityStrength': 0.7
     },
     'imageGenerationConfig': {
         'quality': 'standard',
         'numberOfImages': 1,
         'height': 1024,
         'width': 1024,
         'cfgScale': 8.0
     }
})
#print(enrich_req)

In [16]:
# invoke model to create enriched image
response = bedrock_runtime.invoke_model(body=enrich_req, modelId='amazon.titan-image-generator-v1')
response_body = json.loads(response.get('body').read())
decoded_enriched_image = base64.b64decode(response_body['images'][0])

# Write the decoded image to a file
with open('enriched_boat.png', 'wb') as file:
    file.write(decoded_enriched_image)

#Image.open('enriched_boat.png')